In [79]:
# importamos librerias que vamos a necesitar
import pandas as pd 
import json
import re

Se descomprimen cada uno de los archivos previo a importarlos con pandas, quedamos a disposicion de su archivo json con el cual estaremos realizando el proceso de ETL

In [80]:
# lista vacia donde ingresaremos los datos del archivo
lista=[]
# abrimos archivo json
with open ('dataset_original/output_steam_games.json', 'r', encoding='utf-8') as archivo:
    # creamos un bucle for para iterar e ingresar los datos dentro de la lista vacia
    for iterador in archivo.readlines():
        lista.append(json.loads(iterador))

# guardamos en dataframe
steam_games= pd.DataFrame(lista)
# abrimos dataframe
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


chequeo de valores nulos

In [81]:
# verificamos valores nulos
steam_games.isnull().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [82]:
# eliminamos todas las filas donde todas sus columnas tengan valores nulos y reestablecemos los indices
steam_games=steam_games.dropna(how='all').reset_index(drop=True)

In [83]:
# cantidad de filas que quedaron, despues de eliminar aquellas que tenian todas las columnas con valores nulos
total_filas=steam_games.shape[0]
print(f'total de filas en el dataframe: {total_filas}')

total de filas en el dataframe: 32135


chequeo de valores nulos

In [84]:
# verificamos valores nulos
steam_games.isnull().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [85]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 3.2+ MB


eliminamos las columnas que no utilizaremos con el fin de optimizar el tratamiento de los datos

In [86]:
# procedemos a eliminar las columnas
steam_games.drop(columns=['publisher', 'title', 'url','reviews_url', 'specs', 'early_access', 'tags'], axis=1, inplace=True)
steam_games.reindex()

,genres,app_name,release_date,price,id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,Free To Play,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,Free to Play,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域
4,NaN,Log Challenge,NaN,2.99,773570,NaN
...,...,...,...,...,...,...
32130,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS"""
32131,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada
32132,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich
32133,"[Casual, Indie]",EXIT 2 - Directions,2017-09-02,4.99,658870,"xropi,stev3ns"


In [87]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        28852 non-null  object
 1   app_name      32133 non-null  object
 2   release_date  30068 non-null  object
 3   price         30758 non-null  object
 4   id            32133 non-null  object
 5   developer     28836 non-null  object
dtypes: object(6)
memory usage: 1.5+ MB


In [88]:
# data de la columna 'app_name'
steam_games['app_name']

0             Lost Summoner Kitty
1                       Ironbound
2         Real Pool 3D - Poolians
3                         弹炸人2222
4                   Log Challenge
                   ...           
32130              Colony On Mars
32131    LOGistICAL: South Africa
32132               Russian Roads
32133         EXIT 2 - Directions
32134                 Maze Run VR
Name: app_name, Length: 32135, dtype: object

In [89]:
# verificamos valores nulos en la columna 'app_name'
nulos_app_name= steam_games['app_name'].isnull().sum()

# cantidad total de registros en el dataframe
registros_total= len(steam_games)
print(f'valores nulos en la columna app_name: {nulos_app_name}')
print(f'cantidad total de registros: {registros_total}')

valores nulos en la columna app_name: 2
cantidad total de registros: 32135


In [90]:
# eliminamos valores nulos en la columna 'app_name' ya que los valores nulos solo representan 2 registros para un total de 32135 registros en el dataframe
steam_games.dropna(subset=['app_name'], inplace=True)

In [91]:
# verificamos valores nulos en la columna 'app_name'
steam_games['app_name'].isnull().sum()

0

se normaliza la columna app_name, eliminando caracteres no alfabeticos, espacios adicionales y convirtiendo letras a minuscula

In [92]:
# normalizamos la columna 'app_name', eliminando espacios adicionales, caracteres no alfabeticos y convirtiendo letras a minuscula
def normalize_developer_name(name):
    if isinstance(name, str):  
        cleaned_name = re.sub(r'[^a-zA-Z\s]', '', name)
        normalized_name = re.sub(r'\s+', ' ', cleaned_name.lower().strip())
        return normalized_name
    else:
        return name

steam_games['app_name'] = steam_games['app_name'].apply(normalize_developer_name)
# capitalizamos cada palabra de los valores de 'app_name'
steam_games['app_name']= steam_games['app_name'].str.title()

In [93]:
steam_games['app_name']

0            Lost Summoner Kitty
1                      Ironbound
2           Real Pool D Poolians
3                               
4                  Log Challenge
                  ...           
32130             Colony On Mars
32131    Logistical South Africa
32132              Russian Roads
32133            Exit Directions
32134                Maze Run Vr
Name: app_name, Length: 32133, dtype: object

In [94]:
# cambiamos tipo de dato a la columna 'app_name'
steam_games['app_name']=steam_games['app_name'].astype('string')

In [95]:
# valores unicos de cada fecha
steam_games['release_date'].value_counts()

release_date
2012-10-16    100
2017-08-31     92
2017-09-26     89
2017-06-21     82
2017-07-25     78
             ... 
1988-04-16      1
2013-08-24      1
2011-05-07      1
2010-08-21      1
2018-10-01      1
Name: count, Length: 3582, dtype: int64

# transformacion

extraemos el año de la columna release_date

In [96]:
# extraemos solo el año de release date
steam_games['release_date']=steam_games['release_date'].str.extract(r'(\d{4})')

In [97]:
# rellenamos los valores nulos con 0 con el fin de convertir la columna a tipo de dato int
steam_games['release_date'].fillna('0', inplace=True)
# se convierte la columna 'release_date' a tipo de dato int
steam_games['release_date']= steam_games['release_date'].astype(int)

In [98]:
# valores unicos de la columna 'release_date'
steam_games['release_date'].unique()

array([2018, 2017,    0, 1997, 1998, 2016, 2006, 2005, 2003, 2007, 2002,
       2000, 1995, 1996, 1994, 2001, 1993, 2004, 1999, 2008, 2009, 1992,
       1989, 2010, 2011, 2013, 2012, 2014, 1983, 1984, 2015, 1990, 1988,
       1991, 1985, 1982, 1987, 1981, 1986, 2021, 5275, 2019, 1975, 1970,
       1980])

In [99]:
# excluimos el 0 de nuestros datos
steam_games=steam_games[steam_games['release_date'] !=0]

In [100]:
# renombramos columna 'release_date' a 'release_year'
steam_games= steam_games.rename(columns={'release_date':'release_year'})

In [101]:
# informacion general del dataframe
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29966 entries, 0 to 32133
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        28732 non-null  object
 1   app_name      29966 non-null  string
 2   release_year  29966 non-null  int32 
 3   price         28818 non-null  object
 4   id            29965 non-null  object
 5   developer     28717 non-null  object
dtypes: int32(1), object(4), string(1)
memory usage: 1.5+ MB


In [102]:
# data de la columna 'genres'
steam_games['genres']

0            [Action, Casual, Indie, Simulation, Strategy]
1                     [Free to Play, Indie, RPG, Strategy]
2        [Casual, Free to Play, Indie, Simulation, Sports]
3                              [Action, Adventure, Casual]
5                          [Action, Adventure, Simulation]
                               ...                        
32129                   [Action, Adventure, Casual, Indie]
32130                [Casual, Indie, Simulation, Strategy]
32131                            [Casual, Indie, Strategy]
32132                          [Indie, Racing, Simulation]
32133                                      [Casual, Indie]
Name: genres, Length: 29966, dtype: object

# transformacion

utilizamos la funcion explode para extraer cada valor almacenado en la columna genres y mostrarlos en un nuevo campo conservando su id asociada original

In [103]:
# utilizamos la funcion explode para extraer los valores de la columna 'genres', mostrandolos en un nuevo campo conservando su id asociada al original
steam_games= steam_games.explode('genres')
steam_games=steam_games.dropna(subset=['genres']).reset_index(drop=True)

In [104]:
# convertimos los valores de la columna 'genres' en una cadena separada por comas
steam_games['genres']= steam_games['genres'].astype(str).apply(lambda x: ', '.join(filter(None, x.split(','))))

In [105]:
steam_games

,genres,app_name,release_year,price,id,developer
0,Action,Lost Summoner Kitty,2018,4.99,761140,Kotoshiro
1,Casual,Lost Summoner Kitty,2018,4.99,761140,Kotoshiro
2,Indie,Lost Summoner Kitty,2018,4.99,761140,Kotoshiro
3,Simulation,Lost Summoner Kitty,2018,4.99,761140,Kotoshiro
4,Strategy,Lost Summoner Kitty,2018,4.99,761140,Kotoshiro
...,...,...,...,...,...,...
71209,Indie,Russian Roads,2018,1.99,610660,Laush Dmitriy Sergeevich
71210,Racing,Russian Roads,2018,1.99,610660,Laush Dmitriy Sergeevich
71211,Simulation,Russian Roads,2018,1.99,610660,Laush Dmitriy Sergeevich
71212,Casual,Exit Directions,2017,4.99,658870,"xropi,stev3ns"


In [106]:
# data de la columna 'price'
steam_games['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 3.99, 9.99, 18.99,
       29.99, nan, 10.99, 2.99, 1.59, 14.99, 1.99, 59.99, 8.99, 6.99,
       7.99, 39.99, 'Free', 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25,
       24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99,
       79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 109.99, 149.99, 771.71,
       'Install Now', 21.99, 89.99, 'Play WARMACHINE: Tactics Demo', 0.98,
       139.92, 4.29, 64.99, 'Free Mod', 54.99, 74.99, 'Install Theme',
       0.89, 'Third-party', 0.5, 'Play Now', 299.99, 1.29, 119.99, 3.0,
       15.0, 5.49, 23.99, 49.0, 20.99, 10.93, 1.39,
       'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0, 1.95, 1.5,
       199.0, 189.0, 6.66, 27.99, 129.99, 179.0, 26.99, 399.99, 31.99,
       399.0, 20.0, 40.0, 3.33, 22.99, 320.0, 38.85, 71.7, 995.0, 27.49,
       3.39, 6.0, 19.95, 499.99, 199.99, 16.06, 4.68, 131.4, 44.98,
       202.76, 2.3, 0.95, 172.

hay algunos valores en la columna 'price' que son tipo 'str' , en su gran mayoria haciendo aluciones a que son free, asi que seran reemplazados por 0.0 para posteriormente cambiar el tipo de dato de la columna a float

In [107]:
#reemplazamos 'Free to Play' y otros textos similares por 0 por medio de un diccionario
replacement_dict = {
    'Free to Play': 0.0,
    'Free To Play': 0.0,
    'Play the Demo': 0.0,
    'Install Now': 0.0,
    'Free Mod': 0.0,
    'Play for Free!': 0.0,
    'Play WARMACHINE: Tactics Demo': 0.0,
    'Install Theme': 0.0,
    'Third-party': 0.0,
    'Play Now': 0.0,
    'Free HITMAN™ Holiday Pack': 0.0,
    'Free Movie': 0.0,
    'Free': 0.0,
    'Free Demo': 0.0,
    'Free to Try': 0.0,
    'Free to Use':0.0
}

#aplicamos el reemplazo usando el diccionarip
steam_games['price'].replace(replacement_dict, inplace=True)
# cambiamos tipo de dato de la columna a float
steam_games['price']=steam_games['price'].astype(float)

In [108]:
# contamos nulos de la columna 'price'
nulos_price=steam_games.price.isnull().sum()
print(f'valores nulos columna price: {nulos_price}')
no_nulos_price= steam_games['price'].notnull().sum()
print(f'valores no nulos columna price: {no_nulos_price}')

valores nulos columna price: 2875
valores no nulos columna price: 68339


In [109]:
# proporcion de valores nulos en la columna 'price'
proporcion_nulos_price= steam_games['price'].isnull().sum() / len(steam_games)
print(f'proporcion de valores nulos en price: {proporcion_nulos_price:.2%}')

proporcion de valores nulos en price: 4.04%


ya que la proporcion de valores nulos en la columna price es baja (4.04%), procederemos a eliminar estos registros nulos

In [110]:
# eliminamos valores nulos de la columna 'price'
steam_games= steam_games.dropna(subset=['price'])

In [111]:
# verificamos valores nulos de la columna 'price'
nulos_price=steam_games.price.isnull().sum()
print(f'valores nulos columna price: {nulos_price}')

valores nulos columna price: 0


In [112]:
# data de la columna 'id'
steam_games['id']

0        761140
1        761140
2        761140
3        761140
4        761140
          ...  
71209    610660
71210    610660
71211    610660
71212    658870
71213    658870
Name: id, Length: 68339, dtype: object

In [113]:
# renombramos columna 'id' a 'item_id'
steam_games= steam_games.rename(columns={'id':'item_id'})

In [114]:
# verificamos nulos de la columna 'item_id'
nulos_item_id= steam_games['item_id'].isnull().sum()
# verificamos cantidad total de registros en el dataframe
registros_total= len(steam_games)

print(f'cantidad de nulos en la columna item_id: {nulos_item_id}')
print(f'cantidad total de registros: {registros_total}')


cantidad de nulos en la columna item_id: 2
cantidad total de registros: 68339


In [115]:
# eliminamos valores nulos en la columna 'item_id' ya que los valores nulos solo representan 2 registros para un total de 68,339 registros en el dataframe
steam_games.dropna(subset=['item_id'], inplace=True)

In [116]:
# cambiamos tipo de dato de la columna 'item_id' a int
steam_games['item_id']= steam_games['item_id'].astype(int)

In [117]:
# vemos la data de la columna 'developer'
steam_games['developer']

0                       Kotoshiro
1                       Kotoshiro
2                       Kotoshiro
3                       Kotoshiro
4                       Kotoshiro
                   ...           
71209    Laush Dmitriy Sergeevich
71210    Laush Dmitriy Sergeevich
71211    Laush Dmitriy Sergeevich
71212               xropi,stev3ns
71213               xropi,stev3ns
Name: developer, Length: 68337, dtype: object

In [118]:
# contamos los valores nulos de la columna 'developer'
nulos_developer= steam_games['developer'].isnull().sum()
print(f'valores nulos en la columna developer {nulos_developer}')
# contamos los valores no nulos de la columna 'developer'
no_nulos_developer= steam_games['developer'].notnull().sum()
print(f'valores no nulos de la columna developer: {no_nulos_developer}')

valores nulos en la columna developer 320
valores no nulos de la columna developer: 68017


In [119]:
# proporcion de valores nulos en la columna 'developer'
proporcion_nulo_developer= steam_games['developer'].isnull().sum() / len(steam_games)
print(f'la proporcion de valores nulos de la columna developer es: {proporcion_nulo_developer:.2%}')

la proporcion de valores nulos de la columna developer es: 0.47%


ya que la proporcion de valores nulos de la columna developer es muy baja (0.47%) se proceden a eliminar estos registros nulos

In [120]:
steam_games= steam_games.dropna(subset=['developer']).reset_index(drop=True)
# verificamos los valores nulos de la columna 'developer'
nulos_developer= steam_games['developer'].isnull().sum()
print(f'valores nulos en la columna developer {nulos_developer}')

valores nulos en la columna developer 0


In [121]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68017 entries, 0 to 68016
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        68017 non-null  object 
 1   app_name      68017 non-null  string 
 2   release_year  68017 non-null  int32  
 3   price         68017 non-null  float64
 4   item_id       68017 non-null  int32  
 5   developer     68017 non-null  object 
dtypes: float64(1), int32(2), object(2), string(1)
memory usage: 2.6+ MB


In [122]:
# redondeamos las columnas con caracteristicas mas importantes a criterio
steam_games = steam_games[['item_id','app_name','genres','developer','price','release_year']]

In [123]:
steam_games

,item_id,app_name,genres,developer,price,release_year
0,761140,Lost Summoner Kitty,Action,Kotoshiro,4.99,2018
1,761140,Lost Summoner Kitty,Casual,Kotoshiro,4.99,2018
2,761140,Lost Summoner Kitty,Indie,Kotoshiro,4.99,2018
3,761140,Lost Summoner Kitty,Simulation,Kotoshiro,4.99,2018
4,761140,Lost Summoner Kitty,Strategy,Kotoshiro,4.99,2018
...,...,...,...,...,...,...
68012,610660,Russian Roads,Indie,Laush Dmitriy Sergeevich,1.99,2018
68013,610660,Russian Roads,Racing,Laush Dmitriy Sergeevich,1.99,2018
68014,610660,Russian Roads,Simulation,Laush Dmitriy Sergeevich,1.99,2018
68015,658870,Exit Directions,Casual,"xropi,stev3ns",4.99,2017


# guardamos archivo a clean_steam_games.parquet

In [124]:
steam_games.to_parquet('data/clean_steam_games.parquet')